**Краткое описание шагов**

1. *Анализ данных*

Перед началом анализы текстовые столбцы были переведены в категории для удобства при анализе.

Перед тем как производить работу с моделями для предсказания, необходимо проанализировать данные на предмет отсутствующих данных, выбросов, неинформативных признаков и т.д.

Первым делом я построил корелляционную матрицу, чтобы посмотреть, как фичи связаны между собой. Из неё видно, что с charges, нашим предсказываемым значением, связаны bmi,age,smoker. Также можно заметить, что признаки связаны между собой, хоть и слабо.

![Correlation Matrix](https://i.imgur.com/HCeyuVr.png)


Потом датасет был проанализирован на наличие пропущенных данных с помощью Гистограммы и метода pandas isnull() с выводом процентного соотношения пропущенных данны. Хоть в датасете и указано, что пропущенные данные отстуствуют, в результате чтения, импортирования и т.д., данные могут быть повреждены или утеряны, поэтому их следует проверять.

Результат:


age - 0.0%

sex - 0.0%

bmi - 0.0%

children - 0.0%

smoker - 0.0%

region - 0.0%

charges - 0.0%

num_missing - 0.0%

![Missing data histogram](https://i.imgur.com/kGCdA5m.png)



---


После поиска пропущенных значений я проанализировал датасет на наличие выбросов в некатегориальных признаках, так как выбросы в последствии могут привнести шум в итоговый результат модели, а могут являться вполне реальными данными. В нашем случае есть выбросы, которые, возможно, привносятся индексом массы тела, но являются вполне реальными данными и эти выбросы объяснимы тем, что более толстые люди больше подвержены заболеваниям. Также, выбросы могут появляться из-за разницы регионов, так как в них может быть разная процентная ставка. Как бороться с выбросами обычно решается от задачи к задаче. Их можно удалить, оставить такими же или скорректировать. В моём случае так как удаление выбросов могло привести к потери важной информации, их было решено не трогать.(впоследствии выяснилось, что эти удаление выбросов никак не повлияло на результат)

![BoxPlot](https://i.imgur.com/M2iII0q.png)

![BMI](https://i.imgur.com/Y3USFqX.png)



---



Следующим этапом был поиск неинформативных и нерелевантных признаков. Неинформативные признаки находились путём поиска повторяющихся значений. И если их больше 95%, то признак считается неинформативным. В нашем случае таких признаков не обнаружилось. Нерелевантные признаки ищутся вручную, оценивая с точки зрения человеческого мышления, какие признаки будут влиять на результат. С моей точки зрения этот датасет не обладает нерелевантными признаками, поскольку страховые сборы зависят и от региона, и от здоровья человека, от количества детей и, формально, от того курит он или нет, поскольку курение может вызвать заболевания лёгких.



---



Далее с помощью методов Обратной ликвидации, ExtraTrees и LassoCV я искал наиболее важные фичи. Почему? Большое количество фич сопровождается большим временем работы классификатора или регрессора. Если в ML-моделях это не всегда заметно, то в DL-моделях большое количество фич - это ощутимо сказывается на перформансе. Так же с увеличение числа фич падает точность предсказания, потому что некоторые фичи никак не влияют на результат, но в итоге привносят шум. 

На выходе ExtraTreesRegressor получилось, что наиболее важными признаками являются (чем выше значение, тем важнее признак):

age = 0.12924438

bmi = 0.21532221

smoker = 0.62059197

Обратная ликвидация показала, что все признаки являются важными, кроме sex.

Встроенный метод показал, что все признаки являются важными, кроме sex.

![Embedded method](https://i.imgur.com/76vdfRr.png)



---



Затем string labels были переведены в категории с помощью OneHotEncoder, который исключает мнимую важность одних признаков относительно других в контексте их порядкового номера. После чего все выше перечисленные операции были произведены повторно, чтобы лучше оценить важность признаков. Так, было выявлено, что на результат сильнее влияет то, что человек не курит. В обратной ликвидации видно, что на результате могут сказаться определённые регионы.



![LassoCV](https://i.imgur.com/3dSHNB3.png)


![OneHot Correlation Matrix](https://i.imgur.com/UFwAngo.png)

Отрицательные значения в корреляционной матрице говорят о том, что с увеличением одного значения, второе уменьшается.


---

2. *Построение моделей и предсказание*

Перед тем, как подать данные на вход ML-модели, их нужно нормализовать, потому что модели очень чувствительны к разбросу значений. Они будут лучше предсказывать, если данные будут лежать в одно диапазоне. Чаще всего нормализация используется для логистической регрессии, метода опорных векторов, PCA. Нормализация данных позволяет всем признакам вносить одинаковый вклад (или, что более точно, позволяет добавлять признаки в зависимости от их важности, а не их масштаба). Я использовал RobustScaler, который очень похож на MinMaxScaler за исключением того, что Robust устойчив к выбросам. После нормализации данные были разделены на тренировочную и тестовую выборку. Но чаще всего данные делят на 3 выборки, добавляя валидационную. В этом случае тестовая выборка используется для настройки гиперпараметров,а валидация уже для тестирования полученной модели.

---

Так как задача является задачей Регресии из-за предсказания непрерывной переменной, для сравнения сначала была собрана множественная Линейная регрессия, как базовая регрессионная модель, хотя нет явной линейной зависимости, если посмотреть на распределение признаков.

![Linear Regression](https://i.imgur.com/x1RRS0x.png)


Mean Absolute Error: 3971.0870066054



---


Одним из предположений является то, что оставив только важные признаки, которые содержат большой коэффициент корреляции, модель будет лучше предсказывать. На основании результатов выше перечисленных методов отбора важных признаков, я попробовал подать в модель только BMI,AGE,SMOKE и признаки, которые большинство медотов посчитали важными, добавив к предыдущим трём Children.


![BMI,AGE,SMOKE](https://i.imgur.com/l3K9ENk.png)

Mean Absolute Error: 3980.2856852693103

![BMI,AGE,SMOKER,CHILDREN](https://i.imgur.com/6F5Yd1m.png)

Mean Absolute Error: 3949.200107703656

Действительно, для 4 признаков мы получили меньшую ошибку, однако она все также высока, а для 3 признаков ошибки и вывосле выросли, что говорит о том, что мы, возможно, не учли важную информацию. Как упоминалось выше, из-за небольшокого коэффициента корреляции между признаками и предсказываемым значением, что видно из корелляционной матрицы, мы  получили не очень хорошие резултаты и большие ошибки. Также плохое качество обусловлено наличием категориальных переменных



---


Для того, чтобы улучшить распознавание, было решено использовать регрессионное дерево принятия решений. Оно меньше подвержено ошибкам, лучше справляется с категориальными признаками и данными с ошибками, а также не требуют больших вычислительных затрат. Деревья решений не делают предположения о пространственном распределении. К тому же, они в неявном виде разделяют признаки по важности. 


![Decision Tree](https://i.imgur.com/cxVKDn8.png)

Mean Absolute Error: 2985.2065642388056

Как и в случае с Регрессией выбираем наиболее важные признаками, полученные методами отделения признаков, и сравним с результатами для признаков, которые метод посчитал важными в процессе обучения.


Важные признаки:

age = 0.32324495

bmi = 0.24148527

smoker_no = 0.33018043

---

Получив неплохие результаты, я решил попробовать улучшить результат, взяв за основу один из самых популярных алгоритмов - XGBoost. По производительности он обходит большинство существующих алгоритмов машинного обучения, он также имееет высокую точность за счёт применения ансамбля слабых классификаторов, учитывая в большей степени неправильные предсказания. К тому же он очень гибок к настройке гиперпараметров.

![XGBoost](https://i.imgur.com/D2T6KJa.png)

Mean Absolute Error: 2648.582982425373

Важные признаки XGBoost:

![XGBoost Features](https://i.imgur.com/0qrhotM.png)


В результате получили очень точное предсказание. Если говорить в целом, то важные признаки практически идентичны тем, которые были получены методами выделения признаков, однако разница в XGBoost между children и остальными признаками не такая существенная, чтобы их можно было считать нерелевантными. Переобучив XGboost на 4 наиболее важных признаках, ошибка очень сильно возрасла:

Mean Absolute Error: 4410.471448877992

Это говорит о том, что каждый признак в наборе вносит свой вклад в итоговое предсказываемое значение. С точки зрения изначального анализа датасета на нерелевантные признаки, предположение о том, что все признаки в какой-то степени вносят вклад в итоговое предсказываемое значение оказалось верным. И в конечном счёте выбор этих самых важных признаков зависит от человека, а в работе с большими данными любая ошибка стоит огромного количества времени на доработку, исправление и переобучение.
